<a href="https://colab.research.google.com/github/mordor-ai/M2-MIASHS-DIVERS/blob/main/Correction_TP_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sizes = [512-2*32, 512-32, 512, 512+32]
max_box_per_image = 100
labels=["rien", "masque", "sans masque", "mal"]

anchors = [ 116,90, 156,198, 373,326, #RESOLUTION 8
            30,61, 62, 45, 59,119, #RESOLUTION 16
            10,13, 16,30, 33,23]

In [ ]:
class MyBox:
    def __init__(self, xmin, ymin, xmax, ymax):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
     
    def best_anchors(self, anchors):
        shifted_box = self.shift()
        max_anchor = None
        max_index = -1
        max_iou = -1
        for i, anchor in enumerate(anchors):
            iou    = self.bbox_iou(anchor)
            if max_iou < iou:
                max_anchor = anchor
                max_index  = i
                max_iou    = iou   
        return max_anchor, max_index
        
    def getCenterX(self):
      return 0.5*(self.xmin+self.xmax)
    def getCenterY(self):
        return 0.5*(self.ymin+self.ymax)
    def getCenterW(self):
        return (self.xmax - self.xmin)
    def getCenterH(self):
        return (self.ymax - self.ymin)  
    def shift(self):
        return MyBox(0,0,self.getCenterW(), self.getCenterH())
    def area(self):
        return self.getCenterW() * self.getCenterH()

    def overlap(self, box):
        mini_x = max(self.xmin, box.xmin)
        maxi_x = min(self.xmax, box.xmax)
        intersect_x = max(maxi_x - mini_x, 0)
        
        mini_y = max(self.ymin, box.ymin)
        maxi_y = min(self.ymax, box.ymax)
        intersect_y = max(maxi_y - mini_y, 0) 
        return intersect_x*intersect_y

    def bbox_iou(self, box):
        intersect = self.overlap(box)
        union     = self.area() + box.area() - intersect 
        return intersect / (union + 0.00000000001)


In [ ]:
class Objet:
    def __init__(self, xml_part):
        self.name = str(xml_part.find('name').text) #mask_weared_incorrect without_mask with_mask

        self.xmin = int(xml_part.find('bndbox').find('xmin').text)
        self.xmax = int(xml_part.find('bndbox').find('xmax').text)
        self.ymin = int(xml_part.find('bndbox').find('ymin').text)
        self.ymax = int(xml_part.find('bndbox').find('ymax').text)

    def __str__(self):
        return self.name+" "+str(self.xmin)+" "+str(self.xmax)+" "+str(self.ymin)+" "+str(self.ymax)+" "+str(self.label)


premiers =  [ 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31,37, 41, 43, 47, 53, 59, 61, 67, 71, 73,79, 83, 89, 97, 101, 103, 107, 109, 113, 127,131, 137, 139, 149, 151, 157, 163, 167, 173, 179,181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283,293, 307, 311, 313, 317, 331, 337, 347, 349, 353,359, 367, 373, 379, 383, 389, 397, 401, 409, 419,421, 431, 433, 439, 443, 449, 457, 461, 463, 467,479, 487, 491, 499, 503, 509, 521, 523, 541, 547,557, 563, 569, 571, 577, 587, 593, 599, 601, 607,613, 617, 619, 631, 641, 643, 647, 653, 659, 661,673, 677, 683, 691, 701, 709, 719, 727, 733, 739,743, 751, 757, 761, 769, 773, 787, 797, 809, 811,821, 823, 827, 829, 839, 853, 857, 859, 863, 877,881, 883, 887, 907, 911, 919, 929, 937, 941, 947,953, 967, 971, 977, 983, 997 ]
class ReadXML:
    def __init__(self, ch):
        tree = ET.parse(ch)
        self.root = tree.getroot()
        self.filename=  self.root.find('filename').text
        if "annotations" in ch:
            self.image  = imageio.imread('../images/'+self.filename)[:,:,:3]/255.0
        elif "medical-masks-dataset" in ch:
            self.image  = imageio.imread('../medical-masks-dataset/images/'+self.filename)[:,:,:3]/255.0

        ratio = -1
        if self.image.shape[0]>1024:
            ratio = self.image.shape[0]/1024
        if self.image.shape[1]>1024:
            tmp_ratio = self.image.shape[1]/1024
            if tmp_ratio>ratio:
                ratio = tmp_ratio
        if ratio>0:
             self.image = resize( self.image      , 
                                 ( int(self.image.shape[0]//ratio), int(self.image.shape[1]//ratio)), order=3)
    
    
        dif_axis0=dif_axis1=0
        if (self.image.shape[0]//32 != self.image.shape[0]/32):
            dif_axis0 = (self.image.shape[0]//32+1)*32 - self.image.shape[0]
        if (self.image.shape[1]//32 != self.image.shape[1]/32):
            dif_axis1= (self.image.shape[1]//32+1)*32 - self.image.shape[1]

        self.image = np.pad(self.image, ((0, dif_axis0), (0,dif_axis1), (0,0)), 'constant',  constant_values=(0))
        
        
        
        self.image_zone = np.zeros((self.image.shape[0], self.image.shape[1]), dtype=np.int32)
        self.image_zone[:,:]=1
        self.links={}
        self.valide=True
        
        self.nbMask=self.nbMal=self.nbNone=0
        if (len(self.root.findall('object'))==0):
            print ("prob grave!")
        
        
        #if (len(self.root.findall('object'))>max_box_per_image):
        #    self.valide=False

        self.objs=[]
        for i, ob in enumerate(self.root.findall('object')):
            tmp = Objet(ob)
            
            
            
            '''
            self.image_zone[int(tmp.ymin*ratioY):int(tmp.ymax*ratioY),
                                int(tmp.xmin*ratioX):int(tmp.xmax*ratioX)]*=premiers[i]
            '''
            if ratio>0:
                self.image_zone[int(tmp.ymin//ratio):int(tmp.ymax//ratio),
                                int(tmp.xmin//ratio):int(tmp.xmax//ratio)]*=premiers[i]
            else:
                self.image_zone[tmp.ymin:tmp.ymax,tmp.xmin:tmp.xmax]*=premiers[i]
                

            if tmp.ymin>tmp.ymax or tmp.xmin>tmp.xmax:
                self.valide=False
            
            
            if "annotations" in ch:
                if tmp.name=="with_mask":            
                    self.links[premiers[i]]=2
                    self.nbMask+=1
                elif tmp.name=="mask_weared_incorrect":
                    self.nbMal+=1
                    self.links[premiers[i]]=3
                elif tmp.name=="without_mask":
                    self.links[premiers[i]]=1
                    self.nbNone+=1
                else:
                    print("BUG")

            else:
                if tmp.name=="mask":            
                    self.links[premiers[i]]=2
                    self.nbMask+=1
                elif tmp.name=="poor":
                    self.links[premiers[i]]=3
                    self.nbMal+=1
                elif tmp.name=="none":
                    self.links[premiers[i]]=1
                    self.nbNone+=1
                else:
                    print("BUG")


def crop_resize(img, imglabel, size_square):
    maxi_size = max(img.shape[0], img.shape[1])
    ratio = float(maxi_size)/size_square
    img = resize(img, 
                ( int(min(size_square, img.shape[0]/ratio)), 
                  int(min(size_square, img.shape[1]/ratio))),
                 order=random.choice([0,1,1,1,1,3,3]))
   
    imglabel =tf.image.resize(
                np.expand_dims(imglabel, axis=-1),
        [int(min(size_square, imglabel.shape[0]/ratio)),int(min(size_square, imglabel.shape[1]/ratio))],
        method='nearest').numpy()[:, :, 0]

    dif_axis0=dif_axis1=0
    if (img.shape[0] <size_square):
        dif_axis0 = size_square- img.shape[0] 
    if (img.shape[1] < size_square):
        dif_axis1= size_square- img.shape[1] 

    img      = np.pad(img, ((0, dif_axis0), (0,dif_axis1), (0,0)), 'constant',  constant_values=(0))
    imglabel = np.pad(imglabel, ((0, dif_axis0), (0,dif_axis1)), 'constant',  constant_values=(1))

    return img, imglabel


In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data_link, batch_size=32,  shuffle=True):
        self.downsample = 32
        self.batch_size = batch_size
        self.anchors = [MyBox(0, 0, anchors[2*i], anchors[2*i+1]) for i in range(len(anchors)//2)]

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        keys = [self.indices[k] for k in index]
        return self.__get_data(keys)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

            
    def __get_data(self, keys):
        size = random.choice(sizes)
        net_h, net_w = size, size
        base_grid_h, base_grid_w = net_h//self.downsample, net_w//self.downsample
        x_batch = np.zeros((self.batch_size, net_h, net_w, 3)) 
        t_batch = np.zeros((self.batch_size,1,1,1,max_box_per_image, 4))
        yolo_1 = np.zeros((self.batch_size, 1*base_grid_h,  1*base_grid_w,3,4+1+len(labels))
        yolo_2 = np.zeros((self.batch_size, 2*base_grid_h,  2*base_grid_w,3,4+1+len(labels))
        yolo_3 = np.zeros((self.batch_size, 4*base_grid_h,  4*base_grid_w,3,4+1+len(labels))
        yolos = [yolo_3, yolo_2, yolo_1]

        for id_batch, id_ in enumerate(keys):
            xml = self.data[id_]
            true_box_index=0
            
            
            if random.random()<0.3: 
                mini = int(min(xml.image.shape[0],xml.image.shape[1])*(0.5 + random.random()/2.0))
                px = random.randint(0, xml.image.shape[0] - mini - 1)
                py = random.randint(0, xml.image.shape[1] - mini - 1)
                tmp_img, tmp_label = xml.image[py:py+mini, px:px+mini, :], xml.image_zone[py:py+mini, px:px+mini]
                tmp_img, tmp_label = crop_resize(tmp_img, tmp_label, size)
            else:
                tmp_img, tmp_label = crop_resize(xml.image, xml.image_zone, size)
            x_batch[id_batch] = tmp_img
          
            for j, premier in enumerate(list(xml.links.keys())):
                x = tmp_label%premier==0
                
                if(np.sum(x)>=15):
                    x = np.where(x)

                    box = MyBox(    np.min(x[1]), 
                                    np.min(x[0]),
                                    np.max(x[1]),                                     
                                    np.max(x[0])) 
                    
                    if box.getCenterW()>10 and box.getCenterH()>10:
                        max_anchor, max_index = box.best_anchors(self.anchors)
                        iyolo = max_index // 3
                        yolo = yolos[iyolo]
                        center_x = box.getCenterX()
                        center_x = center_x/net_w * yolo.shape[2]
                        center_y = box.getCenterY()
                        center_y = center_y/net_h * yolo.shape[1]

                        w = np.log((box.getCenterW()+0.00000001 ) / float(max_anchor.xmax))
                        h = np.log((box.getCenterW()+0.00000001 ) / float(max_anchor.xmax))
